In [ ]:
import tensorflow as tf
print('Tensorflow version: ', tf.__version__)

Tensorflow version:  2.3.0


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/My Drive
!ls

In [ ]:
import pandas as pd
import numpy as np

from tensorflow.keras import optimizers
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Input, Embedding, Dropout, GlobalMaxPool1D
from tensorflow.keras.layers import TimeDistributed
from tensorflow.keras.layers import LSTM, Bidirectional
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

from tensorflow.keras import Model, Input
from tensorflow.keras.layers import LSTM, Embedding, Dense
from tensorflow.keras.layers import TimeDistributed, SpatialDropout1D, Bidirectional

import warnings
warnings.filterwarnings("ignore")

from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline

import sklearn
from sklearn.metrics import f1_score
from tensorflow.keras.preprocessing.text import text_to_word_sequence
import pickle

In [ ]:
df = pd.read_excel('train1.xlsx', encoding='latin1')
df = df.fillna(method='ffill')
df.head(20)

,jomle,kalame,barchasb
0,NaN,مدیران,O
1,NaN,چهارمین,O
2,NaN,جشنواره,O
3,NaN,فیلم,O
4,NaN,و,O
5,NaN,عکس,O
6,NaN,فناوری,O
7,NaN,و,O
8,NaN,صنعتی,O
9,NaN,معرفی,O


In [ ]:
print('Unique kalamat in corpus: ', df['kalame'].nunique())
print('Unique barchasbha in corpus: ', df['barchasb'].nunique())

Unique kalamat in corpus:  18071
Unique barchasbha in corpus:  15


In [ ]:
kalame = list(set(df['kalame'].values))
kalame.append('ENDPAD')
num_kalame = len(kalame)

In [ ]:
barchasb = list(set(df['barchasb'].values))
num_barchasb = len(barchasb)

In [ ]:
num_kalame , num_barchasb

(18072, 15)

In [ ]:
class JomleGetter(object):
    def __init__(self, df):
        self.n_sent = 1
        self.df = df
        agg_func = lambda s: [(w, p) for w, p in zip(s['kalame'].values.tolist(),
                                                     s['barchasb'].values.tolist())]
        self.grouped = self.df.groupby('barchasb').apply(agg_func)
        self.jomle = [s for s in self.grouped]

In [ ]:
getter = JomleGetter(df)
jomle = getter.jomle

In [ ]:
jomle[0]

[('پنجم', 'B_DAT'),
 ('دی ماه', 'B_DAT'),
 ('چهارشنبه', 'B_DAT'),
 ('چهارشنبه', 'B_DAT'),
 ('روز', 'B_DAT'),
 ('سال', 'B_DAT'),
 (28, 'B_DAT'),
 ('سال', 'B_DAT'),
 ('اسفند', 'B_DAT'),
 ('چهارم', 'B_DAT'),
 ('ماه', 'B_DAT'),
 ('آوریل', 'B_DAT'),
 ('فروردین', 'B_DAT'),
 ('ماه', 'B_DAT'),
 ('سال', 'B_DAT'),
 ('از', 'B_DAT'),
 (27, 'B_DAT'),
 ('روز', 'B_DAT'),
 ('از', 'B_DAT'),
 (1379, 'B_DAT'),
 ('سال', 'B_DAT'),
 ('سال', 'B_DAT'),
 ('سال', 'B_DAT'),
 ('آوریل', 'B_DAT'),
 ('ژانویه', 'B_DAT'),
 ('ماه', 'B_DAT'),
 ('سه ماهه', 'B_DAT'),
 ('سه ماهه', 'B_DAT'),
 ('سه ماهه', 'B_DAT'),
 ('سال', 'B_DAT'),
 ('ماه', 'B_DAT'),
 ('ماه', 'B_DAT'),
 ('ماه', 'B_DAT'),
 ('نوامبر', 'B_DAT'),
 ('دسامبر', 'B_DAT'),
 ('از', 'B_DAT'),
 ('سال', 'B_DAT'),
 ('آوریل', 'B_DAT'),
 (15, 'B_DAT'),
 (5, 'B_DAT'),
 ('روز', 'B_DAT'),
 ('روز', 'B_DAT'),
 ('پنجشنبه', 'B_DAT'),
 ('روزهای', 'B_DAT'),
 ('امروز', 'B_DAT'),
 ('شامگاه', 'B_DAT'),
 ('جمعه', 'B_DAT'),
 ('پایان', 'B_DAT'),
 ('تیر', 'B_DAT'),
 ('امسال', 'B_DAT'),
 

In [ ]:
kalame2idx = {w: i+1 for i, w in enumerate(kalame)}
barchasb2idx = {t: i for i, t in enumerate(barchasb)}

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [ ]:
max_len = 50
X = [[kalame2idx[w[0]] for w in s] for s in jomle]
X = pad_sequences(X , maxlen = max_len, padding='post', value=num_kalame-1)
print(X)

[[ 1499  8564  8564  8807 16250     8  8564 17660  8564  8564  8564     6
   4252 10130  4471  1109 14749 18047  4400 17660 10823  8564 10823  8564
   8564 13156  8564 18047  8564  8564  7843  1493  1493 16518  5936  1490
   1493 13764  9334  8564  8564  8564  8564 10831 12813  8564  8564  8278
   8564 12813]
 [14214 14214 14214   770   770 13115 13115  8798 11850  5009 17787 16856
  14050  3774  5009 13158  3774 13115 13759 15291   770  3288   770  3288
  15291 15291 13115 15291   770  3288   770 13115 15291 13041  3774  3209
   3774 14864  8652   970  6338  3066   770   770  4341  4341  4341 13759
   1257  4341]
 [   16 13341  8518    26   383    16   161   200 17660   234   190    41
     11   200   250 10823 10823    11 11091   542   383  3797    76    76
   2573   200   206    33    62    37    98  6396    37    51  2398   520
    946   272    55     6    31  2724  2724     6  2004  6396     6   383
      4  2573]
 [ 4078  4078 15364  6877 15364 17378 15364 15715 16028 15369 15715

In [ ]:
X[1]

array([14214, 14214, 14214,   770,   770, 13115, 13115,  8798, 11850,
        5009, 17787, 16856, 14050,  3774,  5009, 13158,  3774, 13115,
       13759, 15291,   770,  3288,   770,  3288, 15291, 15291, 13115,
       15291,   770,  3288,   770, 13115, 15291, 13041,  3774,  3209,
        3774, 14864,  8652,   970,  6338,  3066,   770,   770,  4341,
        4341,  4341, 13759,  1257,  4341], dtype=int32)

In [ ]:
# Convert label to index
y = [[barchasb2idx[w[1]] for w in s] for s in jomle]

# padding
y = pad_sequences(y ,maxlen=max_len , padding='post')

print(y)

[[ 9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9
   9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9
   9  9]
 [13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13
  13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13 13
  13 13]
 [ 6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6
   6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6  6
   6  6]
 [11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11
  11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11 11
  11 11]
 [ 5  5  5  5  5  5  5  5  5  5  5  5  5  5  5  5  5  5  5  5  5  5  5  5
   5  5  5  5  5  5  5  5  5  5  5  5  5  5  5  5  5  5  5  5  5  5  5  5
   5  5]
 [14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14
  14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14
  14 14]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0

In [ ]:
y[1]

array([13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13,
       13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13,
       13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13],
      dtype=int32)

In [ ]:
y[3]

array([11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11,
       11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11,
       11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11],
      dtype=int32)

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=1)

In [ ]:
input_kalame = Input(shape=(max_len, ))
model = Embedding(input_dim = num_kalame, output_dim = max_len, input_length = max_len)(input_kalame)
model = SpatialDropout1D(0.1)(model)
model = Bidirectional(LSTM(units=100, return_sequences=True, recurrent_dropout=0.1))(model)
out = TimeDistributed(Dense(num_barchasb, activation='softmax'))(model)
model = Model(input_kalame, out)
model.summary()

Model: "functional_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 50)]              0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 50, 50)            903600    
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 50, 50)            0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 50, 200)           120800    
_________________________________________________________________
time_distributed_1 (TimeDist (None, 50, 15)            3015      
Total params: 1,027,415
Trainable params: 1,027,415
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [ ]:
early_stopping = EarlyStopping(monitor='val_accuracy', patience=1, verbose=0, mode='max', restore_best_weights=False)

history = model.fit(
    x_train, np.array(y_train),
    validation_split=0.2,
    batch_size = 32,
    epochs = 3,
    #verbose = 1
)

Epoch 1/3


ValueError: ignored